### CARLOS GONZÁLEZ VAN LIEMPT

####          - Actividad Individual 1

##### Código del servidor:

In [ ]:
import socket
import threading
import sys
import pickle
import os

#Solo se puede introducir puertos mayores a 7000 (los anteriores se usan para cosas internas de nuestro sistema)


class Servidor():
	def __init__(self, host=socket.gethostname(), port = input("Escribe el puerto: ")):
		self.clientes = []
		self.mensajes = []
		print("Tu ip es: " + socket.gethostbyname(host))
		self.sock = socket.socket()
		self.sock.bind((str(host), int(port)))
		self.sock.listen(20)
		self.sock.setblocking(False)

		aceptar = threading.Thread(target=self.aceptarC)
		procesar = threading.Thread(target=self.procesarC)


		aceptar.daemon = True
		aceptar.start()

		procesar.daemon = True
		procesar.start()

		
		for thread in threading.enumerate():
			print("Hilo: " + thread.name + "\n" + "Proceso PID: "+ str(os.getpid()) + "\n" + "Daemon: " + str(thread.daemon) +  "\n")
		print("Hilos totales: " + str(threading.activeCount()-1))

		while True:
			msg = input('SALIR = Q\n')
			if msg == 'Q':
				print("**** SERVER CLOSED *****")
				self.sock.close()
				sys.exit()
			else:
				pass

				

	def broadcast(self, msg, cliente):
		self.mensajes.append(pickle.loads(msg))
		print("Los mensajes actuales: " + str(pickle.loads(msg)))
		print("Los mensajes totales: " + str(self.mensajes))
		
		for c in self.clientes:
			try:
				if c != cliente:
					c.send(msg)
			except:
				self.clientes.remove(c)

	def aceptarC(self):
		while True:
			try:
				conn, addr = self.sock.accept()
				print(f"\nConexion aceptada via {conn}\n")
				conn.setblocking(False)
				self.clientes.append(conn)
				for client in self.clientes: 
					data = pickle.dumps(client.username + 'connected')
					self.broadcast(data,client);  
			except:
				pass

	def procesarC(self):
		
		while True:
			if len(self.clientes) > 0:
				for c in self.clientes:
					try:
						data = c.recv(1024)
						if data:
							self.broadcast(data,c)
					except:
						pass
	
		




s = Servidor()

#### --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Código de los clientes:

In [ ]:
import threading
import sys
import socket
import pickle
import os

class Cliente():

	# def __init__(self, host = '10.34.84.147', port=61000):
	def __init__(self, host= input("Escriba la IP: "), port= input("Escriba el puerto: ")):
		self.sock = socket.socket()
		username = input('Username: ')
		self.sock.connect((str(host), int(port)))
		hilo_recv_mensaje = threading.Thread(target=self.recibir)
		hilo_recv_mensaje.daemon = True
		hilo_recv_mensaje.start()
		
		for thread in threading.enumerate():
			print("Hilo: " + thread.name + "\n" + "Proceso PID: "+ str(os.getpid()) + "\n" + "Daemon: " + str(thread.daemon) +  "\n")
		print("Hilos totales: " + str(threading.activeCount()-1))

		self.enviar("<" + username + "> se ha conectado")
		while True:
			# msg = input('\nEscriba texto ? ** Enviar = ENTER ** Abandonar Chat = Q \n')
			msg = input("\n>>\n")
			if msg != 'Q' :
				self.enviar(username + ": " + msg)
			else:
				print(" ****LOGOUT****")
				self.sock.close()
				sys.exit()

	def recibir(self):
		while True:
			try:
				data = self.sock.recv(1024)
				if data:
					print(pickle.loads(data))
			except:
				pass

	def enviar(self, msg):
		self.sock.send(pickle.dumps(msg))
		data = pickle.dumps(msg)
		if data: 
			print(pickle.loads(data))
c = Cliente()
